# Predictions for volume

In [1]:
from enveco.las import *
from enveco.image import *
from enveco.tabular import *

In [2]:
from fastai.data.all import *
from fastai.tabular.all import *
from fastai.callback.progress import ShowGraphCallback

# ANN

In [3]:
preprocessor = EnvecoPreprocessor('../enveco_data/enveco/AV.leaf.on.train.csv', 
                                  '../enveco_data/enveco/AV.leaf.on.val.csv',
                                  '../enveco_data/enveco/AV.leaf.on.test.csv')


Our training set contains a clear outlier for pretty much all variables (v more than 300m³ larger compared to second-largest), so drop it.

In [4]:
preprocessor.train_df[preprocessor.train_df.v > 600]

,sampleplotid,x,y,measurementdate,g,d,h,v,v_ma,v_ku,v_lp,myear,filename_1,a_date_1,a_window_1,filename_2,a_date_2,a_window_2,v_lp_proc,is_valid
75,903136,357090.53,6897053.68,2013,67.5,32.5,28.1,867.35,519.86,331.11,16.38,2013,/wrk/project_ogiir-csc/mml/laserkeilaus/2008_17_automaattinen/2013/20130113_BLOM_ahtari_block3_kesa/1/N4142A2.laz,2013-06-28,C,NaN,NaN,NaN,2,0


In [5]:
preprocessor.train_val_df = preprocessor.train_val_df[preprocessor.train_val_df.v < 600].copy()

Preprocess dataframes and extract lidar features from them. Specify which features are needed:

* height_features:
    * hmax: maximum height of the point cloud
    * hmean: mean height for vegetation pixels
    * hstd: standard deviation for vegetation pixels
    * cv: hstd / hmean
* point_features:
    * vege: proportion of vegetation points
    * ground: proportion of ground points
    * veg_ground_ratio: vege/ground
* intensity_features:
    * imax: maximum intensity for vegetation points
    * imean: mean intensity for vegetation points
    * imed: median intensity for vegetation points
* height_quantiles:
    * H where percentages of vegetation points (0%, 5%,...95%, 100%) were accumulated (e.g. h00, h05...h95, h100)
* point_proportions
    * Proportion of vegetation points having H greater or equal to corresponding percentile of H. H was divided to 10 equal distance fractions
 

In [ ]:
trainval_tb, test_tb = preprocessor.preprocess_lidar(target_col='v', path='../enveco_data/enveco/AV_las/', min_h=1.5,
                                                     height_features=True,
                                                     point_features=True, 
                                                     intensity_features=True, 
                                                     height_quantiles=True,
                                                     point_proportions=True)

Adding height based features
Adding point distribution based features
Adding intensity based features


Put to dataloaders

In [ ]:
dls = trainval_tb.dataloaders(bs=64, y_block=RegressionBlock())

In [ ]:
trainval_tb.train.ys.describe()

In [ ]:
dls.show_batch()

Create `Learner` object. Specify y_range to better control the training, set 1000 m³ as the maximum value for volume.

Loss function is Mean Squared Error, monitor also Root Mean Squared Error, mean-normalized RMSE, Mean Absolute Error, R2Score, bias and mean-scaled bias.

In [ ]:
learn = tabular_learner(dls, metrics=[rmse, rrmse, bias, bias_pct, mae, R2Score()],  y_range=(0,1000))
learn.summary()

In [ ]:
learn.lr_find()

1e-2 seems good learning rate.

In [ ]:
learn.fit_one_cycle(20, max_lr=1e-2, cbs=ShowGraphCallback())

See validation results

In [ ]:
learn.validate()

In [ ]:
reg_interp = RegressionInterpretation.from_learner(learn, ds_idx=1)

In [ ]:
reg_interp.plot_results()
plt.show()

Evaluate test set.

In [ ]:
test_dls = test_tb.dataloaders(y_block=RegressionBlock(), shuffle_train=False, drop_last=False)

In [ ]:
test_interp = RegressionInterpretation.from_learner(learn, dl=test_dls)

In [ ]:
test_interp.plot_results()
plt.show()

# Ensemble of ANNs

Fit several models at once

In [ ]:
ensemble = ANNEnsemble(dls, 
                       y_range=(0,1000), 
                       metrics=[rmse, rrmse, bias, bias_pct, mae, R2Score()], n_models=10)

In [ ]:
ensemble.fit_one_cycle(20, 1e-2)

In [ ]:
res = ensemble.validate()

In [ ]:
res

In [ ]:
ens_int = RegressionInterpretation.from_ensemble(ensemble)

In [ ]:
ens_int.plot_results()
plt.show()

In [ ]:
test_res = ensemble.validate(dl=test_dls[0])
test_res

In [ ]:
test_ens_interp = RegressionInterpretation.from_ensemble(ensemble, dl=test_dls[0])

In [ ]:
test_ens_interp.plot_results()
plt.show()

# Comparison: Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

Below values for max_features and min_samples_leaf should generally work well.

In [ ]:
rf = RandomForestRegressor(n_estimators=500, max_features=0.5, min_samples_leaf=4, oob_score=True)

In [ ]:
rf.fit(trainval_tb.train.xs, trainval_tb.train.ys.values.ravel())

In [ ]:
rf_preds = rf.predict(trainval_tb.valid.xs)

Validation results

In [ ]:
plot_sklearn_regression(rf, trainval_tb.valid.xs, trainval_tb.valid.ys)
plt.show()

Test

In [ ]:
plot_sklearn_regression(rf, test_tb.train.xs, test_tb.train.ys)
plt.show()

## SHAP for RF

Todo comments and info.

In [ ]:
import shap
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(test_tb.train.xs)

Get most influential features

In [ ]:
shap.summary_plot(shap_values, test_tb.train.xs)

In [ ]:
shap.summary_plot(shap_values, test_tb.train.xs, plot_type='bar')